# Formula 1 Analysis

## Load and Explore Data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf

In [2]:
# filepath to datasets
data_filepath = '~/Documents/datasets/Formula 1 Race Data/'

In [3]:
# load various datasets
circuits = pd.read_csv(data_filepath+'circuits.csv', encoding='latin-1')
constructor_results = pd.read_csv(data_filepath+'constructorResults.csv', encoding='latin-1')
constructors = pd.read_csv(data_filepath+'constructors.csv', encoding='latin-1')
constructor_standings = pd.read_csv(data_filepath+'constructorStandings.csv', encoding='latin-1')
drivers = pd.read_csv(data_filepath+'drivers.csv', encoding='latin-1')
driver_standings = pd.read_csv(data_filepath+'driverStandings.csv', encoding='latin-1')
lap_times = pd.read_csv(data_filepath+'lapTimes.csv', encoding='latin-1')
pit_stops = pd.read_csv(data_filepath+'pitStops.csv', encoding='latin-1')
qualifying = pd.read_csv(data_filepath+'qualifying.csv', encoding='latin-1')
races = pd.read_csv(data_filepath+'races.csv', encoding='latin-1')
results = pd.read_csv(data_filepath+'results.csv', encoding='latin-1')
seasons = pd.read_csv(data_filepath+'seasons.csv', encoding='latin-1')
status = pd.read_csv(data_filepath+'status.csv', encoding='latin-1')

Some initial ideas after looking at the data:
* Time series for lap times -- see how lap times change throughout course of a race
* Relation of pit stops -- when is ideal time to box?

In [4]:
# initialize sqldf
pysqldf = lambda q: sqldf(q, globals())

This saves us from having to pass `globals()` as an argument every time we query. With `pysqldf` we can now write queries using SQL syntax, as we are used to. 

In [5]:
# create query for first 10 circuits, 
# only showing name, location and country
q = """SELECT name, location, country
       FROM circuits
       LIMIT 10;"""

# display results
top_10_circuits = pysqldf(q)
top_10_circuits

,name,location,country
0,Albert Park Grand Prix Circuit,Melbourne,Australia
1,Sepang International Circuit,Kuala Lumpur,Malaysia
2,Bahrain International Circuit,Sakhir,Bahrain
3,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain
4,Istanbul Park,Istanbul,Turkey
5,Circuit de Monaco,Monte-Carlo,Monaco
6,Circuit Gilles Villeneuve,Montreal,Canada
7,Circuit de Nevers Magny-Cours,Magny Cours,France
8,Silverstone Circuit,Silverstone,UK
9,Hockenheimring,Hockenheim,Germany


Let's explore circuits a little bit more using SQL.  Would be interesting to see how many circuits are in each country, etc. 

In [6]:
# Display number of circuits per country
# limit results to 10
q = """SELECT country, COUNT(*) as "count"
       FROM circuits
       GROUP BY country
       ORDER BY count DESC
       LIMIT 10;
    """

circuits_per_country = pysqldf(q)
circuits_per_country

,country,count
0,USA,11
1,France,7
2,Spain,6
3,UK,4
4,Portugal,3
5,Japan,3
6,Italy,3
7,Germany,3
8,Canada,3
9,Belgium,3


In [7]:
# further explore USA circuits - surprised there are so many
q = """SELECT name, location, country
       FROM circuits
       WHERE country = "USA"
       ORDER BY location ASC;
    """

usa_circuits = pysqldf(q)
usa_circuits

,name,location,country
0,Circuit of the Americas,Austin,USA
1,Long Beach,California,USA
2,Riverside International Raceway,California,USA
3,Fair Park,Dallas,USA
4,Detroit Street Circuit,Detroit,USA
5,Sebring International Raceway,Florida,USA
6,Indianapolis Motor Speedway,Indianapolis,USA
7,Las Vegas Street Circuit,Nevada,USA
8,Port Imperial Street Circuit,New Jersey,USA
9,Watkins Glen,New York State,USA


In [8]:
# display circuits in each state
q = """SELECT location, COUNT(*) as "count"
       FROM circuits
       WHERE country = "USA"
       GROUP BY location
       ORDER BY count DESC
    """

state_circuits = pysqldf(q)
state_circuits

,location,count
0,California,2
1,Phoenix,1
2,New York State,1
3,New Jersey,1
4,Nevada,1
5,Indianapolis,1
6,Florida,1
7,Detroit,1
8,Dallas,1
9,Austin,1


Now there is a decent understanding of the circuits, especially in the US. Next, look at constructors. 

In [9]:
# query constructors
q = """SELECT name, nationality
       FROM constructors
       LIMIT 10;"""

# display results
top_10_constructors = pysqldf(q)
top_10_constructors


,name,nationality
0,McLaren,British
1,BMW Sauber,German
2,Williams,British
3,Renault,French
4,Toro Rosso,Italian
5,Ferrari,Italian
6,Toyota,Japanese
7,Super Aguri,Japanese
8,Red Bull,Austrian
9,Force India,Indian


In [10]:
# understand how many constructors per nationality
q = """SELECT nationality, COUNT(*) as "count"
       FROM constructors
       GROUP BY nationality
       ORDER BY count DESC
       LIMIT 10;
    """

constructor_nationality = pysqldf(q)
constructor_nationality

,nationality,count
0,British,85
1,American,39
2,Italian,29
3,French,12
4,German,10
5,Japanese,5
6,Swiss,4
7,South African,3
8,Dutch,3
9,Russian,2


Looking at these results, we can see that our data contains a number of seasons worth of data. It may be helpful to look at data by season or year to see how things change from season to season. 

In [11]:
# explore the columns available to us in races
print('Columns in races: {}'.format(list(races.columns)))
print('Columns in circuits: {}'.format(list(circuits.columns)))
print('Columns in constructors: {}'.format(list(constructors.columns)))
print('Columns in drivers: {}'.format(list(drivers.columns)))
print('Columns in seasons: {}'.format(list(seasons.columns)))

Columns in races: ['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url']
Columns in circuits: ['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt', 'url']
Columns in constructors: ['constructorId', 'constructorRef', 'name', 'nationality', 'url', 'Unnamed: 5']
Columns in drivers: ['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob', 'nationality', 'url']
Columns in seasons: ['year', 'url']


In [12]:
# how many seasons are represented
print(f'Seasons represented in dataset: {len(seasons)} seasons')
print(f'Years of data range from {seasons.year.min()} to {seasons.year.max()}')

Seasons represented in dataset: 69 seasons
Years of data range from 1950 to 2018


In [13]:
# Join circuits and races to get race name and date included
q = """
    SELECT circuits.name, circuits.location, circuits.country, races.name, races.year
    FROM circuits JOIN races 
    ON circuits.circuitId = races.circuitId
    """

races_and_circuits = pysqldf(q)
races_and_circuits

,name,location,country,name,year
0,Albert Park Grand Prix Circuit,Melbourne,Australia,Australian Grand Prix,1996
1,Albert Park Grand Prix Circuit,Melbourne,Australia,Australian Grand Prix,1997
2,Albert Park Grand Prix Circuit,Melbourne,Australia,Australian Grand Prix,1998
3,Albert Park Grand Prix Circuit,Melbourne,Australia,Australian Grand Prix,1999
4,Albert Park Grand Prix Circuit,Melbourne,Australia,Australian Grand Prix,2000
...,...,...,...,...,...
992,Sochi Autodrom,Sochi,Russia,Russian Grand Prix,2017
993,Sochi Autodrom,Sochi,Russia,Russian Grand Prix,2018
994,Baku City Circuit,Baku,Azerbaijan,European Grand Prix,2016
995,Baku City Circuit,Baku,Azerbaijan,Azerbaijan Grand Prix,2017


In [50]:
# number of USA circuits by year starting in 2000
q = """
    SELECT c.country, r.year, COUNT(*) as "race_count"
    FROM circuits c 
        LEFT JOIN races r 
        ON c.circuitId = r.circuitId
    WHERE c.country = "USA"
        AND r.year >= 2000
    GROUP BY r.year
    ORDER BY r.year DESC;
    """

kai = pysqldf(q)
kai

,country,year,race_count
0,USA,2018,1
1,USA,2017,1
2,USA,2016,1
3,USA,2015,1
4,USA,2014,1
5,USA,2013,1
6,USA,2012,1
7,USA,2007,1
8,USA,2006,1
9,USA,2005,1


We know the US had 2 races in 2022 and 3 sheduled for 2023 - can tell there has been recent growth, but only one race per year going back 18 years. 